In [15]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import datasets
import numpy as np
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn import set_config
from time import time
from datetime import datetime
from dateutil import parser
from sklearn.model_selection import cross_validate, ParameterGrid, GridSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.preprocessing import OrdinalEncoder, LabelBinarizer
from matplotlib import pyplot as plt
from sklearn import datasets
from sklearn import tree
from sklearn.tree import plot_tree
from sklearn.datasets import load_breast_cancer
from sklearn.metrics import make_scorer, accuracy_score, classification_report

In [16]:
def prep_df(csv):
    global df_original
    df_original = pd.read_csv(csv)
    df = pd.read_csv(csv)
    df["date_today"] = datetime.now()
    df = df.dropna(subset = ["original_estimate_complete_datetime"])
    df["estimate_date_parse"] = df["original_estimate_complete_datetime"].apply(lambda x: parser.parse(str(x)))
    df = df.assign(time_since_estimate =  df['date_today'] - df['estimate_date_parse'])
    df["days_since_estimate"] = df["time_since_estimate"].apply(lambda x: int(str(x).split()[0]))
   
    del df["time_since_estimate"]
    df1 = df[df["suppCnt"] == 0]
    df2 = df[df["suppCnt"] != 0]
    df1["suppInd"] = 0
    df2["suppInd"] = 1
    df = pd.concat([df1,df2], axis = 0)
    df["paid_loss_amount"] = df["paid_loss_amount"].apply(lambda x: abs(x))
    df["primary_point_of_impact"].fillna("Unknown",inplace = True)
    df["total_loss_points"].fillna(999,inplace = True)
    df["insured_fault_pct"].fillna(999,inplace = True)
    global df_w_days
    df_w_days = df
    df = df[["claim_number","company","coverage_grouping","detailed_loss_cause",'driveable_ind',"insured_fault_pct","insured_fault_rating","line_of_business","loss_cause","loss_party_type","loss_state","original_estimate_complete_datetime","payAfterCloseInd","policy_state","policy_type","primary_point_of_impact","saved_vehicle_as_type","secondary_point_of_impact","total_loss_points","vehicle_make","vehicle_year","suppInd","maxNet","minNet","maxFile","minFile","days_since_estimate"]]
    global df_full_data
    df_full_data = df
    df = df[df["days_since_estimate"] >= 90]
    return df

In [17]:
df = prep_df("Supp Prediction.csv")

C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_13748\1398384070.py:3: DtypeWarning: Columns (28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df_original = pd.read_csv(csv)
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_13748\1398384070.py:4: DtypeWarning: Columns (28,35) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(csv)
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_13748\1398384070.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["estimate_date_parse"] = df["original_estimate_complete_datetime"].apply(lambda x: parser.parse(str(x)))
C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_13748\1398384070.py:14: SettingWithCopyWarning: 
A value is trying to be set on 

In [18]:
df = df.set_index("claim_number")
target_name = "suppInd"
Y = df[target_name]
X = df.drop(columns=[target_name,"payAfterCloseInd","days_since_estimate","maxNet","minNet","company","line_of_business","policy_state",])

In [19]:
numerical_columns_selector = selector(dtype_exclude=object)
categorical_columns_selector = selector(dtype_include=object)
numerical_columns = numerical_columns_selector(X)
categorical_columns = categorical_columns_selector(X)
categorical_preprocessor = OneHotEncoder(handle_unknown="ignore")
numerical_preprocessor = StandardScaler()
preprocessor = ColumnTransformer([
    ('one-hot-encoder', categorical_preprocessor, categorical_columns),
    ('standard_scaler', numerical_preprocessor, numerical_columns)])

In [49]:
model = make_pipeline(preprocessor, DecisionTreeClassifier(max_depth = 7))

In [50]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state = 42)

In [51]:
clf = model.fit(X_train, Y_train)


In [52]:
clf.score(X_test, Y_test)

0.8512380349655881

In [53]:
df_under_90 = df_full_data[df_full_data["days_since_estimate"] < 90]
df_under_90["prediction"] = clf.predict(df_under_90.drop(columns=[target_name,"payAfterCloseInd","days_since_estimate","maxNet","minNet","company","line_of_business","policy_state",]))

C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_13748\3283450760.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_under_90["prediction"] = clf.predict(df_under_90.drop(columns=[target_name,"payAfterCloseInd","days_since_estimate","maxNet","minNet","company","line_of_business","policy_state",]))


In [54]:
df_under_90[df_under_90["prediction"] == 1][df_under_90["suppInd"] == 0][["claim_number"]]

C:\Users\TWILLI16\AppData\Local\Temp\ipykernel_13748\4259847201.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  df_under_90[df_under_90["prediction"] == 1][df_under_90["suppInd"] == 0][["claim_number"]]


,claim_number
121,01-004-970117
166,01-004-933214
247,01-004-753826
385,01-004-758484
556,01-004-492433
...,...
286595,01-004-854038
286601,01-004-688060
286611,01-004-827913
286614,01-004-723980


In [42]:
11193

11193